In [1]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:106.0) Gecko/20100101 Firefox/106.0',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.5',
    # 'Accept-Encoding': 'gzip, deflate, br',
    'DNT': '1',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'none',
    'Sec-Fetch-User': '?1',
}
headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:86.0) Gecko/20100101 Firefox/86.0'}

In [3]:
prices = []
names = []
powers = []
gears = []
carbus = []

info_dic = {"Type":[],"Modèle":[],"Marque":[],"Kilométrage":[],"État":[],"Première main":[]
            ,"Origine":[],"Année-Modèle":[],"Nombre de portes":[],"Secteur":[]}
keys = list(info_dic.keys())
linkPrincipal = "https://www.avito.ma/fr/maroc/voitures-%C3%A0_vendre?o="
for i in range(1,3):
    linkPrincipal = linkPrincipal+str(i)
    response = None    
    while response is None or response.status_code != 200:
        response = requests.get(linkPrincipal, headers=headers)
    soup = BeautifulSoup(response.content,"html.parser")
    
    car_divs = soup.find("div",{"class":"sc-1nre5ec-1 fzpnun listing"})
    
    cars = car_divs.find_all("div",{"class":["sc-jejop8-0 epNjzr","sc-jejop8-0 kyKuDw"]})
    links = [car.find("a")["href"] for car in cars]
    
    
    for link in links:
        response = requests.get(link, headers=headers)
        soup = BeautifulSoup(response.content,"html.parser")
        name = soup.find("div",{"class":"sc-1g3sn3w-9 gIlAYt"}).text
        price = soup.find("div",{"class":"sc-1g3sn3w-10 iMiMHk"}).text
        prices.append(price)
        names.append(name)
        infos = soup.find_all("span",{"class":"sc-1x0vz2r-0 kuCwGF"})
        for info in infos:
            text = info.text
            if text in ['Automatique',"Manuelle"]:
                gears.append(text)
            elif text.isalpha() == False:
                powers.append(text)
            else:
                carbus.append(text)
                
        infos_page = soup.find("ol",{"class":"sc-qmn92k-3"})
        infos_data = infos_page.find_all("li",{"class":"sc-qmn92k-1 ldnQxr"})
        existed = []
        
        for info in infos_data:            
            key = info.find("span",{"class":"sc-1x0vz2r-0 brylYP"}).text
            existed.append(key)
            info_dic[key].append(info.find("span",{"class":"sc-1x0vz2r-0 jsrimE"}).text)
                    
        missing = list(set(keys) - set(existed))
        for mis in missing:
            info_dic[mis].append(np.nan)
        

        print(link,"Page : " + str(i))
    info_dic["Name"] = names    
    info_dic["Price"] = prices
    info_dic["Power"] = powers
    info_dic["Gear"] = gears
    info_dic["Fuel"] = carbus
    
df = pd.DataFrame(info_dic)

        
        
    
    

https://www.avito.ma/fr/2_mars/voitures/Audi_a6__49995576.htm 1
https://www.avito.ma/fr/cil/voitures/Stelvio_à_Casablanca_52633231.htm 1
https://www.avito.ma/fr/hay_el_qods/voitures/Audi_Q5_S_Line__49438552.htm 1
https://www.avito.ma/fr/2_mars/voitures/Range_sport_49157449.htm 1
https://www.avito.ma/fr/californie/voitures/Land_Rover_range_rover_sport_Hybride_2021_52475362.htm 1
https://www.avito.ma/fr/centre_ville/voitures/Mercedes_Benz_Class_A200_Pack_AMG__49438511.htm 1
https://www.avito.ma/fr/2_mars/voitures/Megane_1_9_diesel_50305754.htm 1
https://www.avito.ma/fr/californie/voitures/Land_Rover_range_rover_sport_autobiographie__52564169.htm 1
https://www.avito.ma/fr/centre_ville/voitures/Mercedes_Benz_C220_Pack_AMG_49518296.htm 1
https://www.avito.ma/fr/2_mars/voitures/Fort_fiesta_titanium__52128272.htm 1
https://www.avito.ma/fr/californie/voitures/Land_Rover_range_rover_sport_2016_52621512.htm 1
https://www.avito.ma/fr/bournazil/voitures/Chevrolet_captiva_diesel_7_place__52149930.h

In [4]:
df

,Type,Modèle,Marque,Kilométrage,État,Première main,Origine,Année-Modèle,Nombre de portes,Secteur,Name,Price,Power,Gear,Fuel
0,"Voitures d'occasion, à vendre",A6,Audi,200 000 - 249 999,Très bon,Non,WW au Maroc,2008,5,2 Mars,Audi a6,69 000 DH,11 CV,Automatique,Diesel
1,"Voitures d'occasion, à vendre",Stelvio,Alfa Romeo,60 000 - 64 999,Excellent,NaN,WW au Maroc,2019,5,C.I.L,Stelvio à Casablanca,325 000 DH,9 CV,Automatique,Diesel
2,"Voitures d'occasion, à vendre",Q5,Audi,100 000 - 109 999,Excellent,Oui,Importée neuve,2017,5,Hay El Qods,Audi Q5 S-Line,Prix non spécifié,8 CV,Automatique,Diesel
3,"Voitures d'occasion, à vendre",Range Rover Sport,Land Rover,200 000 - 249 999,Très bon,Non,WW au Maroc,2011,5,2 Mars,Range sport,Prix non spécifié,11 CV,Automatique,Diesel
4,"Voitures d'occasion, à vendre",Range Rover Sport,Land Rover,45 000 - 49 999,Excellent,Oui,WW au Maroc,2021,5,Californie,Land Rover range rover sport Hybride 2021,880 000 DH,11 CV,Automatique,Hybride
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,"Voitures d'occasion, à vendre",250,Mercedes-Benz,300 000 - 349 999,Excellent,Non,Dédouanée,1986,5,NaN,mersedes 250 très bon état,30 000 DH,10 CV,Manuelle,Diesel
66,"Voitures d'occasion, à vendre",C4 CACTUS,Citroen,50 000 - 54 999,Très bon,Oui,WW au Maroc,2019,5,NaN,Citroën c4cactus,Prix non spécifié,6 CV,Manuelle,Diesel
67,"Voitures d'occasion, à vendre",XE,Jaguar,55 000 - 59 999,6,Oui,WW au Maroc,2019,5,Californie,Jaguar xe Diesel 2019,295 000 DH,8 CV,Automatique,Diesel
68,"Voitures d'occasion, à vendre",Clio,Renault,95 000 - 99 999,Excellent,NaN,WW au Maroc,2018,NaN,Aïn Chock,Renault bonne état,120 000 DH,6 CV,Automatique,Diesel
